In [ ]:
import pyvista as pv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

PATH = '/Volumes/GoogleDrive/My Drive/Segyio and PyVista'

pv.rcParams['use_panel'] = False

In [ ]:
# Load the Horizons:
df = pd.read_table(os.path.join(PATH, '3DHorizons.xyz'), delim_whitespace=True)
df.head()

In [ ]:
# This is a bit more complicated than needed...
# This dataset has a lot if invalid values so I mask them

invalid_values = [-999.99, ]

def mask_arrays(values):
    if not np.issubdtype(values.dtype, np.number):
        return values
    values = values.copy()
    for val in invalid_values:
        nans = values == val
        if np.any(nans):
            # values = np.ma.masked_where(nans, values)
            values[nans] = np.nan
    return values
        

# Can't have NaNs in spatial refernce
# Since depth has invalid values, use zeros
spatial_ref = ['X', 'Y']
points = np.c_[df[spatial_ref].values, np.zeros(len(df))]

horizon = pv.PolyData(points)
for name in df.keys():
    if name in spatial_ref:
        continue
    values = mask_arrays(df[name].values)
    if np.issubdtype(values.dtype, np.number) and np.all(np.isnan(values)):
        print('Array {} is entirely NaN. Skipping...'.format(name))
        continue
    horizon[name] = values
horizon

In [ ]:
horizon.plot(scalars='Amplitude', clim=[-1, 3])

In [ ]:
cats, hmapping = np.unique(horizon['Horizon'], return_inverse=True)
horizon['Horizon-cats'] = hmapping

# Note: https://github.com/pyvista/pyvista/issues/298
horizon.plot(scalars='Horizon-cats', categories=True)